In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

from f1tenth_gym_jax import make
import jax
import jax.numpy as jnp

# jax.config.update("jax_debug_nans", True)

from f1tenth_gym_jax.envs.utils import Param

In [2]:
num_agents = 10
num_envs = 100

In [53]:
env = make("default_v0", num_agents=num_agents, params=Param(produce_scans=False))

In [54]:
rng = jax.random.PRNGKey(0)
rng, _rng = jax.random.split(rng)
reset_rng = jax.random.split(_rng, num_envs)

In [55]:
obsv, env_state = jax.vmap(env.reset)(reset_rng)

In [56]:
rng, _rng = jax.random.split(rng)
action_rng = jax.random.split(_rng, num_envs*num_agents).reshape((num_agents, num_envs, -1))
actions = {k: jax.vmap(env.action_space(k).sample)(action_rng[i]) for i, k in enumerate(env.agents)}

In [57]:
rng, _rng = jax.random.split(rng)
step_rng = jax.random.split(_rng, num_envs)
obsv, env_state, _, _, info = jax.vmap(env.step)(step_rng, env_state, actions)

In [58]:
def env_init(rng):
    rng, _rng = jax.random.split(rng)
    reset_rng = jax.random.split(_rng, num_envs)
    obsv, env_state = jax.vmap(env.reset)(reset_rng)
    return (env_state, obsv, rng)

In [59]:
def env_step(runner_state, unused):
    env_state, last_obs, rng = runner_state

    rng, _rng = jax.random.split(rng)
    action_rng = jax.random.split(_rng, num_envs*num_agents).reshape((num_agents, num_envs, -1))
    actions = {k: jax.vmap(env.action_space(k).sample)(action_rng[i]) for i, k in enumerate(env.agents)}

    rng, _rng = jax.random.split(rng)
    step_rng = jax.random.split(_rng, num_envs)
    obsv, env_state, _, _, info = jax.vmap(env.step)(step_rng, env_state, actions)

    runner_state = (env_state, obsv, rng)
    return runner_state, None

In [60]:
import time

In [64]:
tick = time.time()
for _ in range(100):
    runner_state = env_init(rng)
    runner_state = jax.lax.scan(env_step, runner_state, None, length=100)
tock = time.time()

In [65]:
sps = 100 * 100 * num_envs / (tock - tick)

In [66]:
sps

272757.36407763645